<a href="https://colab.research.google.com/github/kevin-rn/Grounding-LM/blob/main/abstractive_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Google Colab setup

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%cp -R ./drive/MyDrive/Grounding_LM/ ./

# Packages & Imports

In [4]:
%pip install -q pytorch-lightning
%pip install -q transformers
%pip install -q datasets
%pip install -q openai
# %pip install -q -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.6/720.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 80.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [26]:
from datasets import load_dataset, load_from_disk
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch.optim import AdamW
from tqdm.auto import tqdm
import openai
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint
import random

# Data

## Dataset

In [6]:
# Set Dataset
dataset = "cnn_dailymail"

# Download Dataset
# data = load_dataset("xsum")
# data = load_dataset("cnn_dailymail", "3.0.0")
# data = load_dataset("webis/tldr-17")

# Split Dataset
# df_train = pd.DataFrame(data=data['train'])
# df_val = pd.DataFrame(data=data['validation'])
# df_test = pd.DataFrame(data=data['test'])

# Rename columns for later usage
# df_train.columns = ['text', 'summary', 'id']
# df_val.columns = ['text', 'summary', 'id']
# df_test.columns = ['text', 'summary', 'id']

# Save to data folder (inside Grounding_LM folder)
# df_train.to_csv(f'Grounding_LM/data/{dataset}/train.csv', index=False)
# df_val.to_csv(f'Grounding_LM/data/{dataset}/validation.csv', index=False)
# df_test.to_csv(f'Grounding_LM/data/{dataset}/test.csv', index=False)

In [7]:
# Read Dataset from folder
df_train = pd.read_csv(f'Grounding_LM/data/{dataset}/train.csv')
df_val = pd.read_csv(f'Grounding_LM/data/{dataset}/validation.csv')
df_test = pd.read_csv(f'Grounding_LM/data/{dataset}/test.csv')

df_train.head()

,text,summary,id
0,"LONDON, England (Reuters) -- Harry Potter star...",Harry Potter star Daniel Radcliffe gets £20M f...,42c027e4ff9730fbb3de84c1af0d2c506e41c3e4
1,Editor's note: In our Behind the Scenes series...,Mentally ill inmates in Miami are housed on th...,ee8871b15c50d0db17b0179a6d2beab35065f1e9
2,"MINNEAPOLIS, Minnesota (CNN) -- Drivers who we...","NEW: ""I thought I was going to die,"" driver sa...",06352019a19ae31e527f37f7571c6dd7f0c5da37
3,WASHINGTON (CNN) -- Doctors removed five small...,"Five small polyps found during procedure; ""non...",24521a2abb2e1f5e34e6824e0f9e56904a2b0e88
4,(CNN) -- The National Football League has ind...,"NEW: NFL chief, Atlanta Falcons owner critical...",7fe70cc8b12fab2d0a258fababf7d9c6b5e1262a


## Custom Datamodule

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, data, tokenizer, text_max_len = 512, summary_max_len = 128):
        self.data = data
        self.tokenizer = tokenizer
        self.text_max_len = text_max_len
        self.summary_max_len = summary_max_len

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx: int):
        text = self.data.iloc[idx]['text']
        summary = self.data.iloc[idx]['summary']
        id = self.data.iloc[idx]['id']

        text_encoding = self.tokenizer(
            text,
            max_length=self.text_max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = self.tokenizer(
            summary,
            max_length=self.summary_max_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        labels = summary_encoding['input_ids']
        labels[labels == 0] = -100

        return {
            'text': text,
            'summary': summary,
            'id': id,
            'input_ids': text_encoding['input_ids'],
            'attention_mask': text_encoding['attention_mask'],
            'labels': labels.flatten(),
            'labels_attention_mask': summary_encoding['attention_mask']
        }

In [ ]:
class CustomDataModule(pl.LightningDataModule):
    def __init__(self, df_train, df_val, df_test, tokenizer, batch = 8, text_max_len = 512, summary_max_len = 128):
        super().__init__()
        self.df_train = df_train
        self.df_val = df_val
        self.df_test = df_test
        self.tokenizer = tokenizer
        self.batch = batch
        self.text_max_len = text_max_len
        self.summary_max_len = summary_max_len

    def setup(self, stage=None):
      self.train_dataset = CustomDataset(self.df_train, self.tokenizer, self.text_max_len, self.summary_max_len)
      self.val_dataset = CustomDataset(self.df_val, self.tokenizer, self.text_max_len, self.summary_max_len)
      self.test_dataset = CustomDataset(self.df_test, self.tokenizer, self.text_max_len, self.summary_max_len)

    def collate_fn(self, batch):
      texts = [item['text'] for item in batch]
      summaries = [item['summary'] for item in batch]
      ids = [item['id'] for item in batch]
      text_input_ids = pad_sequence([item['input_ids'].flatten() for item in batch], batch_first=True)
      text_attention_masks = pad_sequence([item['attention_mask'].flatten() for item in batch], batch_first=True)
      labels = pad_sequence([item['labels'] for item in batch], batch_first=True)
      labels_attention_masks = pad_sequence([item['labels_attention_mask'].flatten() for item in batch], batch_first=True)

      return {
          'text': texts,
          'summary': summaries,
          'id': ids,
          'input_ids': text_input_ids,
          'attention_mask': text_attention_masks,
          'labels': labels,
          'labels_attention_mask': labels_attention_masks
      }

    def train_dataloader(self):
      return DataLoader(self.train_dataset, batch_size=self.batch, shuffle=True, num_workers=2, collate_fn=self.collate_fn)

    def val_dataloader(self):
      return DataLoader(self.val_dataset, batch_size=self.batch, shuffle=True, num_workers=2, collate_fn=self.collate_fn)

    def test_dataloader(self):
      return DataLoader(self.test_dataset, batch_size=self.batch, shuffle=True, num_workers=2, collate_fn=self.collate_fn)

## Model

In [ ]:
# class SummaryModel(pl.LightningModule):

#   def __init__(self):
#     super().__init__()
#     self.model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)

#   def forward(self, input_ids, attention_mask, decoder_attention_mask, labels=None):

#     output = self.model(input_ids=input_ids,
#                         attention_mask=attention_mask,
#                         decoder_attention_mask=decoder_attention_mask,
#                         labels=labels,
#     )
#     return output

#   def training_step(self, batch, batch_idx):
#     input_ids = batch["input_ids"]
#     attention_mask = batch["attention_mask"]
#     decoder_attention_mask = batch["labels_attention_mask"]
#     labels = batch["labels"]

#     output = self.forward(input_ids, attention_mask, decoder_attention_mask, labels)
#     loss = output.loss
#     self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#     return loss

#   def validation_step(self, batch, batch_idx):
#     input_ids = batch["input_ids"]
#     attention_mask = batch["attention_mask"]
#     decoder_attention_mask = batch["labels_attention_mask"]
#     labels = batch["labels"]

#     output = self.forward(input_ids, attention_mask, decoder_attention_mask, labels)
#     loss = output.loss
#     self.log("val_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#     return loss

#   def test_step(self, batch, batch_idx):
#     input_ids = batch["input_ids"]
#     attention_mask = batch["attention_mask"]
#     decoder_attention_mask = batch["labels_attention_mask"]
#     labels = batch["labels"]

#     output = self.forward(input_ids, attention_mask, decoder_attention_mask, labels)
#     loss = output.loss
#     self.log("test_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
#     return loss

#   def configure_optimizers(self):
#     return AdamW(self.parameters(), lr=0.0001)

In [ ]:
# N_EPOCHS = 5
# BATCH_SIZE = 32

# tokenizer = T5Tokenizer.from_pretrained('t5-base')
# data_module = CustomDataModule(df_train, df_val, df_test, tokenizer, batch=BATCH_SIZE)
# model = SummaryModel()

# checkpoint_callback = ModelCheckpoint(dirpath="checkpoints", filename="best_checkpoints", save_top_k=1,
#                                          verbose=True, monitor="val_loss", mode="min")

# trainer = pl.Trainer(callbacks=checkpoint_callback, max_epochs=N_EPOCHS, accelerator="gpu", enable_progress_bar=True)

In [ ]:
# trainer.fit(model, data_module)

# trained_model = SummaryModel.load_from_checkpoint(trainer.checkpoint_callback.best_model_path)
# trained_model.freeze()

# Fine-Tuned models

### Pretrained T5 model

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sysresearch101/t5-large-finetuned-xsum-cnn")
model = AutoModelForSeq2SeqLM.from_pretrained("sysresearch101/t5-large-finetuned-xsum-cnn")

### BART

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-xsum")

### GPT-2

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Ayham/xlmroberta_gpt2_summarization_xsum")
model = AutoModelForSeq2SeqLM.from_pretrained("Ayham/xlmroberta_gpt2_summarization_xsum")

## Generate Summaries

In [ ]:
data_module = CustomDataModule(df_train, df_val, df_test, tokenizer, batch=8)
data_module.setup()

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

summaries, ids = [], []

with torch.no_grad():
    for batch in tqdm(data_module.test_dataloader()):
        summary_ids = model.generate(
          input_ids = batch['input_ids'].to(device),
          attention_mask=batch['attention_mask'].to(device),
          max_length=150,
          num_beams=2,
          repetition_penalty=2.5,
          length_penalty=1.0,
          early_stopping=True
      )

        summary_text = [tokenizer.decode(sum_id, skip_special_tokens=True, clean_up_tokenization_spaces=True) for sum_id in summary_ids]
        summaries.extend(summary_text)
        ids.extend(batch['id'])

  0%|          | 0/1417 [00:00<?, ?it/s]

In [ ]:
data = {'id': ids, 'generated': summaries}
df_new = pd.DataFrame(data)
df_new.to_csv("summary_output.csv", index=False)

In [ ]:
%cp summary_output.csv ./drive/MyDrive/Grounding_LM/results/

# OPEN-AI Models


In [32]:
openai.api_key = 'sk-'

### text-davinci-003
Using In-Context learning where we should deal with the 4096-token limit

In [37]:
def format_prompt(example_text, input_text):
    prompt = """System: You are an extractive summarizer that follows the output pattern.
            User: The following examples are successful extractive summarization instances: """ + example_text + """.
            Please summarize the following document. Document: + """ + input_text + """.
            The summary should contain at most 5 sentences that summarize the document."""
    return prompt

def generate_summary(input_list, example_text):
  sum_dict = {"text":[],"summary":[]}

  for input_text in tqdm(input_list):
    sum_dict['text'].append(input_text)
    prompt = format_prompt(example_text, input_text)

    response = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0.5,
      max_tokens=256,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0
    )
    sum_df["description"].append(response['choices'][0]['text'])
  sum_df = pd.DataFrame(sum_dict)
  sum_df.to_csv("davinci003_xsum.csv",index=False)

In [35]:
text_inputs = df_test['text'].tolist()

random_rows = df_train.sample(n=5)
example_prompt = ""
for _, row in random_rows.iterrows():
  example_prompt += "input: " + row['text'] + ",output: " + row['summary'] + "\n\n"

print(example_prompt)

input: Egypt has stepped up its crackdown on the smuggling tunnels linking Egypt to the Gaza Strip which began following the ousting of President Mohamed Morsi in July, Hamas officials claim. Security forces are using bulldozers and dynamite to demolish the network of tunnels which are used to transfer cheap fuel and commercial goods into the Palestinian territory. They are also occasionally used by fighters to move weapons. Scroll down for video . A Palestinian tunnel worker is lowered on a rope into a smuggling tunnel dug beneath the Gaza-Egypt border in the southern Gaza Strip . Pro-Palestine Campaigners say the crackdown is having dire consequences on people living in Gaza who are now effectively locked in. Abu Ziyad, a Palestinian tunnel operator told the BBC: 'The number of tunnels was over 1,000. Today only a few hundred remain with the ongoing demolitions. 'Now the number of workers in the tunnel industry has dropped.' As a result of the clampdown the costs of food and building

In [39]:
# generate_summary(text_inputs, example_prompt)

In [1]:
%cp davinci003_xsum.csv ./drive/MyDrive/Grounding_LM/results/

cp: cannot stat 'davinci003_xsum.csv': No such file or directory


### get-3.5-turbo

In [ ]:
def generate_summary(text):
  prompt = text + "\n\nTl;dr"
  response = openai.Completion.create(
    model="get-3.5-turbo",
    prompt=prompt,
    temperature=0.7,
    max_tokens=150,
    top_p=0.9,
    frequency_penalty=0.0,
    presence_penalty=1
  )
  summary_text = response['choices'][0]['text'].strip()
  return summary_text

In [ ]:
df_test['get-3.5-turbo'] = df['text'].apply(generate_summary)

In [ ]:
df.to_csv('summary_output.csv', index=False)